# Pràctica II - Perceptron amb Scikit-learn

En la sessió anterior hem implementat i entrenat un perceptró des de zero, analitzant pas a pas el seu funcionament intern. Ara, donarem un pas més i treballarem amb la implementació ja disponible a la llibreria scikit-learn, que ens permetrà concentrar-nos en el procés d’aplicació del model a un conjunt de dades real.

Per tal d’il·lustrar-ho, farem servir el problema del Titanic, que ja coneixeu del curs anterior. L’objectiu és predir si un passatger va sobreviure o no al naufragi a partir d’algunes de les seves característiques (edat, sexe, classe del bitllet, etc.).

Tasques a realitzar

1. Carregar i preparar les dades
   - Importar el conjunt de dades del Titanic.
   - Seleccionar les variables més rellevants i realitzar els preprocessaments necessaris (tractament de valors NaNs, codificació de variables categòriques i normalització).
   - Dividir les dades en conjunts d’entrenament i validació.

2. Definir i entrenar el model
   - Utilitzar la classe Perceptron de ``sklearn.linear_model``.
   - Ajustar el model amb les dades d’entrenament.
   - Utilitzar el perceptró implementat la sessió anterior per comparar resultats.

3. Avaluar el rendiment
   - Calcular la *accuracy* sobre les dades de validació.
   - Analitzar la matriu de confusió per entendre millor els encerts i errors del model.
   - Comparar els resultats de ``scikit-learn`` amb la vostra implementació prèvia.

In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split

Podeu trobar les dades del Titanic a [Kaggle](https://www.kaggle.com/competitions/titanic/overview).

In [89]:
df = pd.read_csv('titanic.csv')

## Neteja de dades

### Codificació de variables categòriques

Farem servir la codificació *one-hot* per a les variables categòriques, ``pandas`` té una funció que ens ho facilita: ``get_dummies()``.

In [90]:
dummies = pd.get_dummies(df)

### Eliminació de les columnes sense informació

Hi ha algunes columnes que no aporten informació rellevant per a la predicció, com ara el nom del passatger o el número del bitllet. Les hem d'eliminar.

In [91]:
dummies.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2,1,1,38.0,1,0,71.2833,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,1,3,26.0,0,0,7.9250,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,4,1,1,35.0,1,0,53.1000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,5,0,3,35.0,0,0,8.0500,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Normalitzam les dades

Hem de normalitzar les dades per tal que totes les característiques tinguin la mateixa escala. Això és especialment important per a algorismes com el perceptró, que són sensibles a la magnitud de les característiques.

In [92]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
dummies[['Fare', 'Age']] = min_max_scaler.fit_transform(dummies[['Fare', 'Age']])

dummies.head()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,0.271174,1,0,0.014151,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2,1,1,0.472229,1,0,0.139136,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,1,3,0.321438,0,0,0.015469,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,4,1,1,0.434531,1,0,0.103644,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,5,0,3,0.434531,0,0,0.015713,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Tractament de valors NaN

Els valors NaN els hem de tractar abans d'entrenar el model. Podem optar per eliminar les files amb valors NaN o bé imputar-los amb la mitjana, mediana o moda de la columna. Pandas ens ofereix diverses funcions per fer-ho. Per exemple, podem utilitzar ``fillna()`` per imputar valors, o ``dropna()`` per eliminar files amb valors NaN.

In [93]:
dummies[dummies.isna().any(axis=1)]



,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
5,6,0,3,NaN,0,0,0.016510,False,False,False,...,False,False,False,False,False,False,False,False,True,False
17,18,1,2,NaN,0,0,0.025374,False,False,False,...,False,False,False,False,False,False,False,False,False,True
19,20,1,3,NaN,0,0,0.014102,False,False,False,...,False,False,False,False,False,False,False,True,False,False
26,27,0,3,NaN,0,0,0.014102,False,False,False,...,False,False,False,False,False,False,False,True,False,False
28,29,1,3,NaN,0,0,0.015379,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,NaN,0,0,0.014110,False,False,False,...,False,False,False,False,False,False,False,True,False,False
863,864,0,3,NaN,8,2,0.135753,False,False,False,...,False,False,False,False,False,False,False,False,False,True
868,869,0,3,NaN,0,0,0.018543,False,False,False,...,False,False,False,False,False,False,False,False,False,True
878,879,0,3,NaN,0,0,0.015412,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [94]:
#decido eliminar porque si les asigno el valor promedio de edad, puede ser que esto contamine, a lo mejor estos
#tenga una edad promedia más joven o más vieja que el promedio de todo el barco.
dummies = dummies.dropna(subset=['Age'])
dummies[dummies.isna().any(axis=1)]



,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S


### Divisió del conjunt de dades

Separarem les dades en un conjunt d'entrenament i un de validació. Això ens permetrà avaluar el rendiment del model en dades que no ha vist durant l'entrenament. S'utilitza la funció ``train_test_split()`` de ``sklearn.model_selection``.

In [95]:
X = dummies.drop('Survived', axis=1)
y = dummies['Survived']
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)","Name_Abelson, Mr. Samuel",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,3,0.271174,1,0,0.014151,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2,1,0.472229,1,0,0.139136,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,3,0.321438,0,0,0.015469,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,4,1,0.434531,1,0,0.103644,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,5,3,0.434531,0,0,0.015713,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [96]:
from sklearn.linear_model import Perceptron
model = Perceptron()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)


In [97]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6083916083916084
              precision    recall  f1-score   support

           0       0.61      1.00      0.76        87
           1       0.00      0.00      0.00        56

    accuracy                           0.61       143
   macro avg       0.30      0.50      0.38       143
weighted avg       0.37      0.61      0.46       143



C:\Users\liuji\PycharmProjects\aa_2526\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\liuji\PycharmProjects\aa_2526\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\liuji\PycharmProjects\aa_2526\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

In [98]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from Perceptron import Perceptron as Perceptron1

X1 = dummies.drop('Survived', axis=1)
y1 = dummies['Survived']


p1 = Perceptron1()
p1.fit(X1, y1)  # Ajusta els pesos
y_prediction = p1.predict(X)  # Prediu

from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

- Bucle de samples 0


KeyError: 0